# 1.
Заново обучите лучшую модель и сделайте прогноз;
Создайте датафрейм с важностью, выведите его в порядке убывания важности признаков;
Какие признаки наиболее важны для этой модели?

Подсказка:
Когда вы перебирали цикл, последняя обученная модель m могла быть не той, которую вы выбрали, поэтому обучите её заново и сделайте прогноз на тестовой выборке.

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# прочитаем из csv-файла данные с описанием автомобилей и их потреблением, распечатаем его размер и первые 5 строк
cars = pd.read_csv('../datasets/auto_cons.csv')

# уберём строки с пустыми значениями из набора данных
cars.dropna(inplace=True)

# сохраним датафрейм с учётом преобразования признаков
cars = pd.get_dummies(cars)

# разделим наши данные на признаки (матрица X) и целевую переменную (y)
X = cars.drop(columns=['Расход топлива'])
y = cars['Расход топлива']

# разделяем модель на обучающую и валидационную выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# создадим объект класса StandardScaler и применим его к обучающей выборке
scaler = StandardScaler()
X_train_st = scaler.fit_transform(
    X_train
)  # обучаем scaler и одновременно трансформируем матрицу для обучающей выборки

# применяем стандартизацию к матрице признаков для тестовой выборки
X_test_st = scaler.transform(X_test)

# задайте список моделей
models = [
    Lasso(),
    Ridge(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
]

# функция, которая вычисляет MAPE
def mape(y_true, y_pred):
    y_error = y_true - y_pred
    y_error_abs = [abs(i) for i in y_error]
    perc_error_abs = y_error_abs / y_true
    return perc_error_abs.sum() / len(y_true)


# функция, которая принимает на вход модель и данные и выводит метрики
def make_prediction(m, X_train, y_train, X_test, y_test):
    model = m
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(
        'MAE:{:.2f} MSE:{:.2f} MAPE:{:.2f} R2:{:.2f} '.format(
            mean_absolute_error(y_test, y_pred),
            mean_squared_error(y_test, y_pred),
            mape(y_test, y_pred),
            r2_score(y_test, y_pred),
        )
    )


# напишите цикл, который выводит метрики по списку моделей
for i in models:
    print(i)
    make_prediction(
        m=i,
        X_train=X_train_st,
        y_train=y_train,
        X_test=X_test_st,
        y_test=y_test,
    )

# обучим финальную модель
final_model = models[-1]
final_model.fit(X=X_train_st, y=y_train)
y_pred = final_model.predict(X=X_test_st)

# создадим датафрейм с именами признаков и их важностью и выведем его по убыванию важности
fi_df = pd.DataFrame(data={'feature': X.columns, 'importance': final_model.feature_importances_})
fi_df.sort_values('importance', ascending=False)


Lasso()
MAE:1.49 MSE:3.68 MAPE:0.15 R2:0.76 
Ridge()
MAE:0.97 MSE:1.79 MAPE:0.09 R2:0.88 
DecisionTreeRegressor()
MAE:1.14 MSE:2.48 MAPE:0.11 R2:0.84 
RandomForestRegressor()
MAE:0.92 MSE:1.60 MAPE:0.09 R2:0.89 
GradientBoostingRegressor()
MAE:0.91 MSE:1.56 MAPE:0.09 R2:0.90 


,feature,importance
3,Вес,0.619419
2,Мощность,0.151007
5,Год выпуска,0.104173
1,Рабочий объем двигателя,0.101135
4,Ускорение,0.019944
8,Производитель_US,0.003417
0,Кол-во цилиндров,0.000729
7,Производитель_Europe,0.000093
6,Производитель_Asia,0.000083
